Viewing the Jupyter Notebooks from nbviewer is encouraged because GitHub is still not fully integrated with the Jupyter Notebook:
http://nbviewer.jupyter.org/github/suyunu/GA-FSS/blob/master/ga-fss.ipynb

# Genetic Algorithm on Flow Shop Scheduling

In this project, we tried to solve Flow Shop Scheduling Problrm (FSSP) with Genetic Algorithm (GA). Before I start doing anything on the problem, I made a literature survey and found these 2 papers:
* Murata, Tadahiko, Hisao Ishibuchi, and Hideo Tanaka. "Genetic algorithms for flowshop scheduling problems." Computers & Industrial Engineering 30.4 (1996): 1061-1071
* Reeves, Colin R. "A genetic algorithm for flowshop sequencing." Computers & operations research 22.1 (1995): 5-13.

These 2 papers have done lots of good optmization tests for the parameters and obtained good results. So I took pieces form both of the papers:
* Murata et al.
    * General Structure
    * Crossover 
    * Mutation
* Reeves
    * Selection


## Flow Shop Scheduling

There are n machines and m jobs. Each job contains exactly n operations. The i-th operation of the job must be executed on the i-th machine. No machine can perform more than one operation simultaneously. For each operation of each job, execution time is specified.
Operations within one job must be performed in the specified order. The first operation gets executed on the first machine, then (as the first operation is finished) the second operation on the second machine, and so until the n-th operation. Jobs can be executed in any order, however. Problem definition implies that this job order is exactly the same for each machine. The problem is to determine the optimal such arrangement, i.e. the one with the shortest possible total job execution makespan.

## Solution Representation

I used a simple permutation representation.

The string "ABCDEF" represents a job sequence where "Job A" is processed first, then "Job B" is processed, and so on. If the string "ABCADE" is generated by genetic operators such as crossover and mutation, this string is not a feasible solution of the flow shop scheduling problem because the job "A" appears twice in the string and the job "F" does not appear. Therefore the string used in the flow shop scheduling problem should be the permutation of given jobs.

## Genetic Algorithm

In this part I will explain the genetic operators such as crossover and mutation as well as the selection mechanism for the flow shop scheduling problem.

### Pseudocode

1. **(Initialization)** Randomly generate an initial population $P_1$ of $N_{pop}$ strings (i,e., $N_{pop}$ solutions).
2. **(Selection)** Select $N_{pop}$ pairs of strings from a current population according to the selection probability.
3. **(Crossover)** Apply crossover operator to each of the selected pairs in Step 2 to generate $N_{pop}$ solutions with the crossover probability $P_c$. If the crossover operator is not applied to the selected pair, one of the selected solutions remains as a new string.
4. **(Mutation)** Apply mutation operator to each of the generated $N_{pop}$ strings with the mutation probability $P_m$ (we assign the mutation probability not to each bit but to each string).
5. **(Elitist Update)** Randomly remove one string from the current population and add the best string in the previous population to the current one.
6. **(Termination)** If a prespecified stopping condition is satisfied, stop this algorithm. Otherwise, return to Step 2

### Initialization

I randomly generated $N_{pop}$ number of solutions for the initial population. Suprisingly after lots of test, $N_{pop} = 3$ gave the best results.

### Selection

The usual method (for maximization) is to measure relative fitness as the ratio of the value of a given chromosome to the population mean. However, in minimization problems, we need to modify this so that low-valued chromosomes are the "good" ones. One approach is to define fitness as $(v_{max} - v)$, but of course we are unlikely to know what $v_{max}$ is. We could use the largest value found so far as a surrogate for $v_{max}$, but in practice this method was not very good at distinguishing between chromosomes. In essence, if the population consists of many chromosomes whose fitness values are relatively close, the resolution between "good" and "bad" ones is not of a high order. This approach was therefore abandoned, in favour of a simple ranking mechanism. Sorting the chromosomes is a simple task, and updating the list is also straightforward. The selection of parents was then made in accordance with the probability distribution

$ p(k) = \frac{2k}{M(M+1)} $

where $k$ is the $k^{th}$ chromosome in ascending order of fitness (i.e. descending order of makespan). This implies that the median value has a chance of $\frac{1}{M}$ of being selected, while the $M^{th}$ (the fittest) has a chance of $\frac{2}{M+1}$, roughly twice that of the median.

We choose $N_{pop}$ pairs of strings from the current population according to this selection probability. So at the end we will have $N_{pop}$ number of children reproduced from those selected pairs. 

### Crossover

According to Murata et al. Two-point crossover gives overall better results than other crossover techniques. Also having a crossover probability $P_c = 1$ gives the best results. So we applied crossover to every parent.

In Two-point crossover, two points are randomly selected for dividing one parent. The jobs outside the selected two points are always inherited from one parent to the child, and the other jobs are placed in the order of their appearance in the other parent.

![Crossover](crossover.PNG)


### Mutation

Again according to Murata et al. Shift change gives overall better results than other mutation techniques. Also having a mutation probability $P_m = 1$ gives the best results. So we applied mutation to every child.

In shift change mutation, a job at one position is removed and put at another position. Then all other jobs shifted accordingly. The two positions are randomly selected. 

![Mutation](mutation.PNG)


### Elitist Update Strategy

We applied all the operations to $N_{pop}$ pairs of strings (parents). So we ended up with $N_{pop}$ number of children. As a last step we randomly remove one string from the current population and add the best string in the previous population to the current one. Then we continue our processes with this newly generated population.


### Termination

Total number of evaluations/generations used as a stopping condition. The total number of generations was specifed as to be inversely proportional to the population size $N_{pop}$. For example, when the total number of evaluations was $10,000$, the total number of generations was specified as $\frac{10,000}{N_{pop}}$

 choose the number of generation to stop the program as $10000$

## Summary

* **Population size:** $N_{pop} = 3$
* **Crossover probability:** $P_c = 1.0$
* **Crossover:** Two-point crossover
* **Mutation probability:** $P_m = 1.0$
* **Mutation:** Shift change mutation
* **Stopping Condition:** Generation $= 10000$

### Importing required libraries

In [10]:
import numpy as np
import math
import time
import random
import itertools
import queue

### Reading data
Reading input from document and initializing variables.
You can change dataset variable to 1, 2 or 3 to use different input sets.

In [17]:
# Dataset number. 1, 2 or 3
dataset = "3"

if dataset == "1":
    optimalObjective = 4534
elif dataset == "2":
    optimalObjective = 920
else:
    optimalObjective = 1302

filename = "data" + dataset + ".txt"
f = open(filename, 'r')
l = f.readline().split()

# number of jobs
n = int(l[0])

# number of machines
m = int(l[1])

# ith job's processing time at jth machine 
cost = []
    
for i in range(n):
    temp = []
    for j in range(m):
        temp.append(0)
    cost.append(temp)
    
for i in range(n):
    line = f.readline().split()
    for j in range(int(len(line)/2)):
        cost[i][j] = int(line[2*j+1])
    
f.close()

### Genetic Algorithm Operators and Helper Functions

We described all this functions above, except how do we calculate the objective function, which is actually an essential part of this code. 

#### Objective Value Calculation

We want to find out the time when the last machine finishes processing of the last job in the list, which is makespan.

We first defined a variable called time to follow the times of the processed jobs and to calculate when a job will be finished when it enters a machine. Then we put those calculations into a priority queue, which sorts the job-machine pairs according to the completion time. In each iteration we take the next item in the priority queue, which has 3 variables:
* **Time:** the current time of the system.
* **Machine:** The machine id that finished the process of the given job.
* **Job:** The job id that was processed by the given machine.

Then we check two processes:

1. If the next machine for this job is available, then put this job and the next machine into the priority queue with the updated time.
2. If the next machine for this job is not available, then put this job into the waiting queue of the next machine.
3. If there is any job in the waiting queue of this machine, take the first job from the waiting queue and ready it for the processing, putting that job with the machine into the priority queue.

In [4]:
def initialization(Npop):
    pop = []
    for i in range(Npop):
        p = list(np.random.permutation(n))
        while p in pop:
            p = list(np.random.permutation(n))
        pop.append(p)
    
    return pop

def calculateObj(sol):
    qTime = queue.PriorityQueue()
    
    qMachines = []
    for i in range(m):
        qMachines.append(queue.Queue())
    
    for i in range(n):
        qMachines[0].put(sol[i])
    
    busyMachines = []
    for i in range(m):
        busyMachines.append(False)
    
    time = 0
    
    job = qMachines[0].get()
    qTime.put((time+cost[job][0], 0, job))
    busyMachines[0] = True
    
    while True:
        time, mach, job = qTime.get()
        if job == sol[n-1] and mach == m-1:
            break
        busyMachines[mach] = False
        if not qMachines[mach].empty():
                j = qMachines[mach].get()
                qTime.put((time+cost[j][mach], mach, j))
                busyMachines[mach] = True
        if mach < m-1:
            if busyMachines[mach+1] == False:
                qTime.put((time+cost[job][mach+1], mach+1, job))
                busyMachines[mach+1] = True
            else:
                qMachines[mach+1].put(job)
            
    return time
        

def selection(pop):
    popObj = []
    for i in range(len(pop)):
        popObj.append([calculateObj(pop[i]), i])
    
    popObj.sort()
    
    distr = []
    distrInd = []
    
    for i in range(len(pop)):
        distrInd.append(popObj[i][1])
        prob = (2*(i+1)) / (len(pop) * (len(pop)+1))
        distr.append(prob)
    
    parents = []
    for i in range(len(pop)):
        parents.append(list(np.random.choice(distrInd, 2, p=distr)))
    
    return parents

def crossover(parents):
    pos = list(np.random.permutation(np.arange(n-1)+1)[:2])
    
    if pos[0] > pos[1]:
        t = pos[0]
        pos[0] = pos[1]
        pos[1] = t
    
    child = list(parents[0])
    
    for i in range(pos[0], pos[1]):
        child[i] = -1
    
    p = -1
    for i in range(pos[0], pos[1]):
        while True:
            p = p + 1
            if parents[1][p] not in child:
                child[i] = parents[1][p]
                break
    
    return child


def mutation(sol):
    pos = list(np.random.permutation(np.arange(n))[:2])
    
    if pos[0] > pos[1]:
        t = pos[0]
        pos[0] = pos[1]
        pos[1] = t
    
    remJob = sol[pos[1]]
    
    for i in range(pos[1], pos[0], -1):
        sol[i] = sol[i-1]
        
    sol[pos[0]] = remJob
    
    return sol
        
def elitistUpdate(oldPop, newPop):
    bestSolInd = 0
    bestSol = calculateObj(oldPop[0])
    
    for i in range(1, len(oldPop)):
        tempObj = calculateObj(oldPop[i])
        if tempObj < bestSol:
            bestSol = tempObj
            bestSolInd = i
            
    rndInd = random.randint(0,len(newPop)-1)
    
    newPop[rndInd] = oldPop[bestSolInd]
    
    return newPop

# Returns best solution's index number, best solution's objective value and average objective value of the given population.
def findBestSolution(pop):
    bestObj = calculateObj(pop[0])
    avgObj = bestObj
    bestInd = 0
    for i in range(1, len(pop)):
        tObj = calculateObj(pop[i])
        avgObj = avgObj + tObj
        if tObj < bestObj:
            bestObj = tObj
            bestInd = i
            
    return bestInd, bestObj, avgObj/len(pop)

### Searching for the Best Population Size

I've tried 3 different population sizes for 3 datasets. You can find the results below. Obviously and suprisingly population size = 3 gives the best results.

#### Dataset 1

In [14]:
Npops = [3,3,3,3,3,5,5,5,5,5,10,10,10,10,10]
print("Dataset " + str(dataset))
for npp in Npops:
    # Number of population
    Npop = npp
    # Probability of crossover
    Pc = 1.0
    # Probability of mutation
    Pm = 1.0
    # Stopping number for generation
    stopGeneration = 10000

    # Start Timer
    t1 = time.clock()

    # Creating the initial population
    population = initialization(Npop)

    # Run the algorithm for 'stopGeneration' times generation
    for i in range(stopGeneration):
        # Selecting parents
        parents = selection(population)
        childs = []

        # Apply crossover
        for p in parents:
            r = random.random()
            if r < Pc:
                childs.append(crossover([population[p[0]], population[p[1]]]))
            else:
                if r < 0.5:
                    childs.append(population[p[0]])
                else:
                    childs.append(population[p[1]])

        # Apply mutation 
        for c in childs:
            r = random.random()
            if r < Pm:
                c = mutation(c)

        # Update the population
        population = elitistUpdate(population, childs)

        #print(population)
        #print(findBestSolution(population))

    # Stop Timer
    t2 = time.clock()

    # Results Time

    bestSol, bestObj, avgObj = findBestSolution(population)
    G = 100 * (bestObj-optimalObjective) / optimalObjective
    timePassed = (t2-t1)
    
    print("PopSize: " + str(Npop) + " - ObjVal: " + str(bestObj) + " - AvgObj: " + "%.2f" %avgObj + " - %Gap: " + "%.2f" %G + " - CPU Time (s): " + "%.2f" %timePassed)

Dataset 1
PopSize: 3 - ObjVal: 4534 - AvgObj: 4929.33 - %Gap: 0.00 - CPU Time (s): 24.80
PopSize: 3 - ObjVal: 4534 - AvgObj: 4840.33 - %Gap: 0.00 - CPU Time (s): 25.36
PopSize: 3 - ObjVal: 4534 - AvgObj: 5002.00 - %Gap: 0.00 - CPU Time (s): 26.22
PopSize: 3 - ObjVal: 4534 - AvgObj: 5154.33 - %Gap: 0.00 - CPU Time (s): 26.35
PopSize: 3 - ObjVal: 4534 - AvgObj: 4836.00 - %Gap: 0.00 - CPU Time (s): 25.44
PopSize: 5 - ObjVal: 4534 - AvgObj: 5066.80 - %Gap: 0.00 - CPU Time (s): 43.37
PopSize: 5 - ObjVal: 4534 - AvgObj: 5356.20 - %Gap: 0.00 - CPU Time (s): 43.84
PopSize: 5 - ObjVal: 4534 - AvgObj: 5663.20 - %Gap: 0.00 - CPU Time (s): 43.72
PopSize: 5 - ObjVal: 4534 - AvgObj: 4850.20 - %Gap: 0.00 - CPU Time (s): 44.11
PopSize: 5 - ObjVal: 4534 - AvgObj: 5050.60 - %Gap: 0.00 - CPU Time (s): 43.64
PopSize: 10 - ObjVal: 4534 - AvgObj: 5530.40 - %Gap: 0.00 - CPU Time (s): 86.21
PopSize: 10 - ObjVal: 4534 - AvgObj: 5839.50 - %Gap: 0.00 - CPU Time (s): 86.90
PopSize: 10 - ObjVal: 4534 - AvgObj: 567

#### Dataset 2

In [16]:
Npops = [3,3,3,3,3,5,5,5,5,5,10,10,10,10,10]
print("Dataset " + str(dataset))
for npp in Npops:
    # Number of population
    Npop = npp
    # Probability of crossover
    Pc = 1.0
    # Probability of mutation
    Pm = 1.0
    # Stopping number for generation
    stopGeneration = 10000

    # Start Timer
    t1 = time.clock()

    # Creating the initial population
    population = initialization(Npop)

    # Run the algorithm for 'stopGeneration' times generation
    for i in range(stopGeneration):
        # Selecting parents
        parents = selection(population)
        childs = []

        # Apply crossover
        for p in parents:
            r = random.random()
            if r < Pc:
                childs.append(crossover([population[p[0]], population[p[1]]]))
            else:
                if r < 0.5:
                    childs.append(population[p[0]])
                else:
                    childs.append(population[p[1]])

        # Apply mutation 
        for c in childs:
            r = random.random()
            if r < Pm:
                c = mutation(c)

        # Update the population
        population = elitistUpdate(population, childs)

        #print(population)
        #print(findBestSolution(population))

    # Stop Timer
    t2 = time.clock()

    # Results Time

    bestSol, bestObj, avgObj = findBestSolution(population)
    G = 100 * (bestObj-optimalObjective) / optimalObjective
    timePassed = (t2-t1)
    
    print("PopSize: " + str(Npop) + " - ObjVal: " + str(bestObj) + " - AvgObj: " + "%.2f" %avgObj + " - %Gap: " + "%.2f" %G + " - CPU Time (s): " + "%.2f" %timePassed)

Dataset 2
PopSize: 3 - ObjVal: 920 - AvgObj: 1001.00 - %Gap: 0.00 - CPU Time (s): 41.74
PopSize: 3 - ObjVal: 929 - AvgObj: 1002.33 - %Gap: 0.98 - CPU Time (s): 42.15
PopSize: 3 - ObjVal: 920 - AvgObj: 1055.00 - %Gap: 0.00 - CPU Time (s): 45.23
PopSize: 3 - ObjVal: 920 - AvgObj: 964.67 - %Gap: 0.00 - CPU Time (s): 40.75
PopSize: 3 - ObjVal: 937 - AvgObj: 1028.67 - %Gap: 1.85 - CPU Time (s): 39.75
PopSize: 5 - ObjVal: 931 - AvgObj: 1052.40 - %Gap: 1.20 - CPU Time (s): 64.36
PopSize: 5 - ObjVal: 937 - AvgObj: 1185.20 - %Gap: 1.85 - CPU Time (s): 64.04
PopSize: 5 - ObjVal: 937 - AvgObj: 1074.00 - %Gap: 1.85 - CPU Time (s): 64.76
PopSize: 5 - ObjVal: 926 - AvgObj: 1076.00 - %Gap: 0.65 - CPU Time (s): 67.21
PopSize: 5 - ObjVal: 920 - AvgObj: 1063.60 - %Gap: 0.00 - CPU Time (s): 66.68
PopSize: 10 - ObjVal: 937 - AvgObj: 1177.00 - %Gap: 1.85 - CPU Time (s): 123.27
PopSize: 10 - ObjVal: 931 - AvgObj: 1166.00 - %Gap: 1.20 - CPU Time (s): 122.97
PopSize: 10 - ObjVal: 929 - AvgObj: 1139.80 - %Gap:

#### Dataset 3

In [18]:
Npops = [3,3,3,3,3,5,5,5,5,5,10,10,10,10,10]
print("Dataset " + str(dataset))
for npp in Npops:
    # Number of population
    Npop = npp
    # Probability of crossover
    Pc = 1.0
    # Probability of mutation
    Pm = 1.0
    # Stopping number for generation
    stopGeneration = 10000

    # Start Timer
    t1 = time.clock()

    # Creating the initial population
    population = initialization(Npop)

    # Run the algorithm for 'stopGeneration' times generation
    for i in range(stopGeneration):
        # Selecting parents
        parents = selection(population)
        childs = []

        # Apply crossover
        for p in parents:
            r = random.random()
            if r < Pc:
                childs.append(crossover([population[p[0]], population[p[1]]]))
            else:
                if r < 0.5:
                    childs.append(population[p[0]])
                else:
                    childs.append(population[p[1]])

        # Apply mutation 
        for c in childs:
            r = random.random()
            if r < Pm:
                c = mutation(c)

        # Update the population
        population = elitistUpdate(population, childs)

        #print(population)
        #print(findBestSolution(population))

    # Stop Timer
    t2 = time.clock()

    # Results Time

    bestSol, bestObj, avgObj = findBestSolution(population)
    G = 100 * (bestObj-optimalObjective) / optimalObjective
    timePassed = (t2-t1)
    
    print("PopSize: " + str(Npop) + " - ObjVal: " + str(bestObj) + " - AvgObj: " + "%.2f" %avgObj + " - %Gap: " + "%.2f" %G + " - CPU Time (s): " + "%.2f" %timePassed)

Dataset 3
PopSize: 3 - ObjVal: 1302 - AvgObj: 1363.00 - %Gap: 0.00 - CPU Time (s): 57.17
PopSize: 3 - ObjVal: 1302 - AvgObj: 1390.00 - %Gap: 0.00 - CPU Time (s): 57.09
PopSize: 3 - ObjVal: 1302 - AvgObj: 1341.00 - %Gap: 0.00 - CPU Time (s): 56.90
PopSize: 3 - ObjVal: 1302 - AvgObj: 1302.00 - %Gap: 0.00 - CPU Time (s): 57.42
PopSize: 3 - ObjVal: 1302 - AvgObj: 1346.33 - %Gap: 0.00 - CPU Time (s): 56.91
PopSize: 5 - ObjVal: 1302 - AvgObj: 1521.20 - %Gap: 0.00 - CPU Time (s): 94.06
PopSize: 5 - ObjVal: 1302 - AvgObj: 1403.40 - %Gap: 0.00 - CPU Time (s): 93.96
PopSize: 5 - ObjVal: 1302 - AvgObj: 1398.40 - %Gap: 0.00 - CPU Time (s): 93.88
PopSize: 5 - ObjVal: 1302 - AvgObj: 1497.00 - %Gap: 0.00 - CPU Time (s): 93.97
PopSize: 5 - ObjVal: 1323 - AvgObj: 1453.00 - %Gap: 1.61 - CPU Time (s): 93.81
PopSize: 10 - ObjVal: 1302 - AvgObj: 1610.50 - %Gap: 0.00 - CPU Time (s): 182.90
PopSize: 10 - ObjVal: 1302 - AvgObj: 1619.90 - %Gap: 0.00 - CPU Time (s): 189.32
PopSize: 10 - ObjVal: 1302 - AvgObj: 1

### Main Solver and Results

In [19]:
# Number of population
Npop = 3
# Probability of crossover
Pc = 1.0
# Probability of mutation
Pm = 1.0
# Stopping number for generation
stopGeneration = 10000

# Start Timer
t1 = time.clock()

# Creating the initial population
population = initialization(Npop)

# Run the algorithm for 'stopGeneration' times generation
for i in range(stopGeneration):
    # Selecting parents
    parents = selection(population)
    childs = []
    
    # Apply crossover
    for p in parents:
        r = random.random()
        if r < Pc:
            childs.append(crossover([population[p[0]], population[p[1]]]))
        else:
            if r < 0.5:
                childs.append(population[p[0]])
            else:
                childs.append(population[p[1]])
    
    # Apply mutation 
    for c in childs:
        r = random.random()
        if r < Pm:
            c = mutation(c)

    # Update the population
    population = elitistUpdate(population, childs)
    
    #print(population)
    #print(findBestSolution(population))

# Stop Timer
t2 = time.clock()
    
# Results Time

bestSol, bestObj, avgObj = findBestSolution(population)
    
print("Population:")
print(population)
print() 

print("Solution:")
print(population[bestSol])
print() 

print("Objective Value:")
print(bestObj)
print()

print("Average Objective Value of Population:")
print("%.2f" %avgObj)
print()

print("%Gap:")
G = 100 * (bestObj-optimalObjective) / optimalObjective
print("%.2f" %G)
print()

print("CPU Time (s)")
timePassed = (t2-t1)
print("%.2f" %timePassed)

Population:
[[3, 15, 11, 13, 14, 8, 17, 7, 0, 12, 9, 19, 5, 16, 18, 6, 4, 10, 2, 1], [3, 15, 11, 13, 14, 8, 17, 7, 0, 5, 12, 9, 10, 16, 19, 18, 6, 4, 2, 1], [3, 15, 11, 13, 14, 8, 0, 5, 19, 17, 18, 16, 12, 9, 7, 6, 4, 10, 2, 1]]

Solution:
[3, 15, 11, 13, 14, 8, 17, 7, 0, 12, 9, 19, 5, 16, 18, 6, 4, 10, 2, 1]

Objective Value:
1302

Average Objective Value of Population:
1339.67

%Gap:
0.00

CPU Time (s)
62.60
